This project is about PDF Comparison
we convert pdf in byte format and 
store database name 'pdf' table name 'PdfComparison' and by using PDF ID we compare pdf and store into table in column ComparedPdfData

In [2]:
###########################################################################################################
############################  About project  ##############################################################
########################  PDF Comparison      #####################################################
###########################   Date: 29aug2024   ######################################################
#########################################################################################################

############################################################
#################### Import libraries ####################
##############  required libraries   ####################
########### import fitz ..........for PDF processing  ##################
##############  from PIL import Image ......... Pillow Python Imaging Library (PIL)  ###########
############   import io  ...........Input output libraries  ##################
##############  import numpy as np..........numerical computing in Python. ################
############## structural_similarity as compare_ssim.........  image processing  ######################
##############   pyodbc..........  For MSSQL connection    ###################
############################################################################


import fitz              # PyMuPDF for PDF processing #  pip install PyMuPDF
from PIL import Image    # pip install Pillow Python Imaging Library (PIL)
import io                # Input output libraries
import numpy as np       # library for numerical computing in Python. 
import cv2               # image processing tasks, such as converting images to grayscale.
from skimage.metrics import structural_similarity as compare_ssim   #image processing in Python. 
import pyodbc            # For MSSQL connection

#################### Database connection details  ########################################

username = 'sa'
password = 'Apvike123$'
server = 'APVIKE-AI\\SQLEXPRESS2022'
database = 'pdf'


#################### Connection string ############################################################

connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

#################### Connect to MSSQL Database ############################################################

def connect_to_db():
    return pyodbc.connect(connection_string)


####################################################################################################
#################### following code is used for reading the  two pdfs to be compared   ####################
#################### Fetch PDF byte data from SQL Server table ########################################
####################################################################################################

def fetch_pdfs_from_db(pdf_id1, pdf_id2):
    with connect_to_db() as conn:
        cursor = conn.cursor()
        cursor.execute("""
            SELECT [Pdf1Data], [Pdf1Name], [ComparedPdfName]
            FROM [pdf].[dbo].[PdfComparison]
            WHERE [Id] IN (?, ?)
        """, (pdf_id1, pdf_id2))
        rows = cursor.fetchall()

    if len(rows) != 2:
        raise ValueError("Two PDFs must be provided for comparison.")

    pdf1_data, pdf1_name, compared_pdf_name = rows[0]
    pdf2_data, pdf2_name, _ = rows[1]  

    return pdf1_data, pdf1_name, pdf2_data, pdf2_name, compared_pdf_name

####################################################################################################
#################### following code is used for writing the result of pdf comparison   ####################
#################### Insert compared PDF data into SQL Server table ####################
####################################################################################################



def insert_pdfs(pdf1_data, compared_pdf_data, pdf1_name, compared_pdf_name):
    with connect_to_db() as conn:
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO PdfComparison (Pdf1Name, Pdf1Data, ComparedPdfName, ComparedPdfData)
            VALUES (?, ?, ?, ?)
        """, (pdf1_name, pdf1_data, compared_pdf_name, compared_pdf_data))
        conn.commit()

####################################################################################################
#################### Extract text and style information from a PDF page  ####################
####################################################################################################

def extract_text_with_style(page):
    blocks = page.get_text("dict")["blocks"]
    text_data = []
    prev_bottom = None
    for block in blocks:
        if block['type'] == 0:
            for line in block["lines"]:
                line_spacing = None
                if prev_bottom is not None:
                    line_spacing = line["bbox"][1] - prev_bottom
                prev_bottom = line["bbox"][3]
                for span in line["spans"]:
                    text_data.append({
                        "text": span["text"].replace(" ", ""),
                        "bbox": span["bbox"],
                        "size": span["size"],
                        "flags": span["flags"],
                        "font": span["font"],
                        "line_spacing": line_spacing
                    })
    return text_data

####################################################################################################
#################### Compare extracted text data from two pages  ####################
####################################################################################################


def compare_texts(text_data1, text_data2):
    differences = []
    for data1, data2 in zip(text_data1, text_data2):
        if (data1["text"] != data2["text"] or 
            data1["size"] != data2["size"] or 
            data1["flags"] != data2["flags"] or 
            data1["font"] != data2["font"] or 
            data1["line_spacing"] != data2["line_spacing"]):
            differences.append((data1, data2))
    return differences

################################################################################
#################### Highlight text differences on a PDF page ####################
################################################################################

def highlight_text_differences(page, text_differences):
    for diff in text_differences:
        data1, _ = diff
        bbox = fitz.Rect(data1["bbox"])
        page.draw_rect(bbox, color=(1, 0, 0), width=2)

################################################################################
#################### Highlight differences between two images  ####################
################################################################################

def highlight_differences(img1, img2):
    gray1 = cv2.cvtColor(np.array(img1), cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(np.array(img2), cv2.COLOR_BGR2GRAY)

    score, diff = compare_ssim(gray1, gray2, full=True)
    diff = (diff * 255).astype("uint8")

    thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    highlighted_img = np.array(img2)
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(highlighted_img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    return Image.fromarray(highlighted_img), len(contours)


####################################################################################################
#################### Convert images to PDF bytes  ########################################
####################################################################################################

def images_to_pdf_bytes(images):
    pdf_bytes_io = io.BytesIO()
    images[0].save(pdf_bytes_io, save_all=True, append_images=images[1:], format='PDF')
    return pdf_bytes_io.getvalue()

####################################################################################################
#################### Main function to compare PDFs and highlight differences ####################
####################################################################################################

def main(pdf_id1, pdf_id2):
    try:
        # Fetch PDFs from the database
        pdf1_data, pdf1_name, pdf2_data, pdf2_name, compared_pdf_name = fetch_pdfs_from_db(pdf_id1, pdf_id2)

        # Convert byte data to fitz document
        pdf1 = fitz.open(stream=io.BytesIO(pdf1_data))
        pdf2 = fitz.open(stream=io.BytesIO(pdf2_data))

        num_pages1 = len(pdf1)
        num_pages2 = len(pdf2)

        min_pages = min(num_pages1, num_pages2)
        max_pages = max(num_pages1, num_pages2)

        if num_pages1 != num_pages2:
            print(f"Warning: PDFs have different number of pages. Comparing up to {min_pages} pages.")
            print(f"Additional pages in the larger PDF will be noted as differences.")

        images1 = [Image.open(io.BytesIO(pdf1[i].get_pixmap().tobytes())) for i in range(num_pages1)]
        images2 = [Image.open(io.BytesIO(pdf2[i].get_pixmap().tobytes())) for i in range(num_pages2)]

        highlighted_images = []
        total_differences = 0

        for page_num in range(min_pages):
            page1 = pdf1.load_page(page_num)
            page2 = pdf2.load_page(page_num)

            text_data1 = extract_text_with_style(page1)
            text_data2 = extract_text_with_style(page2)
            text_differences = compare_texts(text_data1, text_data2)
            highlight_text_differences(page1, text_differences)
            
            total_differences += len(text_differences)
            
            img1 = images1[page_num]
            img2 = images2[page_num]

            highlighted_img, num_image_differences = highlight_differences(img1, img2)
            highlighted_images.append(highlighted_img)
            total_differences += num_image_differences

        if num_pages1 != num_pages2:
            additional_pages = max_pages - min_pages
            print(f"Additional {additional_pages} page(s) in the larger PDF will be noted as differences.")

        # Convert highlighted images to PDF bytes
        compared_pdf_data = images_to_pdf_bytes(highlighted_images)

        # Insert the compared PDF data into the database
        insert_pdfs(pdf1_data, compared_pdf_data, pdf1_name, "compared_pdf_name")

        print(f"Total differences found: {total_differences}")
        print(f"Compared PDF data saved to database with name {"compared_pdf_name"}.")
    
    except Exception as e:
        print(f"An error occurred: {e}")

#################### Specify the IDs for the PDF comparison ####################

pdf_id1 = 1  # Replace with the actual first ID
pdf_id2 = 2 # Replace with the actual second ID

#################### Run the main function ####################

main(pdf_id1, pdf_id2)

Total differences found: 7
Compared PDF data saved to database with name compared_pdf_name.
